In [11]:
import logging
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Set up Chrome options
def setup_driver():
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("start-maximized")
    options.add_argument("--disable-infobars")
    options.add_argument("--disable-extensions")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option("useAutomationExtension", False)
    logger.info("Chrome WebDriver setup complete.")
    return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Scrape hotel details from the listing page
def scrape_hotel_details(driver):
    hotels_data = []
    last_scroll_height = 0

    while True:
        logger.info("Scrolling through the page to load more hotels.")
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        hotel_elements = driver.find_elements(By.CSS_SELECTOR, "div.WlYyy.cPsXC.dTqpp")

        for hotel in hotel_elements:
            hotel_data = {}
            try:
                hotel_name = hotel.find_element(By.CSS_SELECTOR, "div.biGQs._P.fiohW.rRtyp").text
                hotel_data["Hotel Name"] = hotel_name
                logger.info(f"Scraped hotel name: {hotel_name}")
            except Exception as e:
                hotel_data["Hotel Name"] = None
                logger.error(f"Error scraping hotel name: {e}")

            try:
                hotel_location = hotel.find_element(By.CSS_SELECTOR, "a.BMQDV._F.Gv.wSSLS.SwZTJ.FGwzt.suezE").text
                hotel_data["Location"] = hotel_location
                logger.info(f"Scraped hotel location: {hotel_location}")
            except Exception as e:
                hotel_data["Location"] = None
                logger.error(f"Error scraping hotel location: {e}")

            try:
                hotel_rating = hotel.find_element(By.CSS_SELECTOR, "span.IvARn").text
                hotel_data["Rating"] = hotel_rating
                logger.info(f"Scraped hotel rating: {hotel_rating}")
            except Exception as e:
                hotel_data["Rating"] = None
                logger.error(f"Error scraping hotel rating: {e}")

            hotels_data.append(hotel_data)

        # Scroll down to load more hotels
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Adjust the delay based on your internet speed

        # Check if we've reached the bottom of the page
        new_scroll_height = driver.execute_script("return document.body.scrollHeight;")
        if new_scroll_height == last_scroll_height:
            logger.info("Reached the bottom of the page. Stopping scrolling.")
            break
        last_scroll_height = new_scroll_height

    return hotels_data

# Scrape reviews for each hotel
def scrape_hotel_reviews(driver, hotel_data):
    reviews_data = []
    hotel_name = hotel_data.get("Hotel Name", "Unknown Hotel")
    logger.info(f"Navigating to hotel page for reviews: {hotel_name}")

    try:
        driver.find_element(By.LINK_TEXT, hotel_name).click()
        time.sleep(2)  # Allow time for the page to load

        # Switch to the new window/tab
        driver.switch_to.window(driver.window_handles[1])
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        review_elements = soup.select("div.yCeTE")

        for review in review_elements:
            review_data = hotel_data.copy()
            try:
                review_data["User Name"] = review.select_one("a.ui_header_link").text
                logger.info(f"Scraped user name: {review_data['User Name']}")
            except Exception as e:
                review_data["User Name"] = None
                logger.error(f"Error scraping user name: {e}")

            try:
                review_data["Review Rating"] = review.select_one("span.ui_bubble_rating")["class"][1].split("_")[1]
                logger.info(f"Scraped review rating: {review_data['Review Rating']}")
            except Exception as e:
                review_data["Review Rating"] = None
                logger.error(f"Error scraping review rating: {e}")

            try:
                review_data["Review Given"] = review.select_one("q").text
                logger.info(f"Scraped review text: {review_data['Review Given']}")
            except Exception as e:
                review_data["Review Given"] = None
                logger.error(f"Error scraping review text: {e}")

            try:
                review_data["Date of Review"] = review.select_one("span.ratingDate")["title"]
                logger.info(f"Scraped date of review: {review_data['Date of Review']}")
            except Exception as e:
                review_data["Date of Review"] = None
                logger.error(f"Error scraping date of review: {e}")

            reviews_data.append(review_data)

        # Close the current tab and switch back
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        logger.info(f"Completed scraping reviews for {hotel_name}")

    except Exception as e:
        logger.error(f"Error navigating to hotel reviews for {hotel_name}: {e}")

    return reviews_data

# Main function to run the scraping
def main():
    logger.info("Starting the web scraping process.")
    driver = setup_driver()
    url = 'https://www.tripadvisor.in/TravelersChoice-Hotels-cLuxury-g1'
    driver.get(url)
    time.sleep(5)  # Allow time for the page to load
    logger.info(f"Loaded URL: {url}")

    hotels_data = scrape_hotel_details(driver)
    all_data = []

    for hotel in hotels_data:
        logger.info(f"Processing hotel: {hotel.get('Hotel Name', 'Unknown Hotel')}")
        reviews = scrape_hotel_reviews(driver, hotel)
        if reviews:
            all_data.extend(reviews)
        else:
            all_data.append(hotel)

    # Save to Excel
    df = pd.DataFrame(all_data)
    output_file = 'tripadvisor_hotels_reviews.xlsx'
    df.to_excel(output_file, index=False, engine='openpyxl')
    driver.quit()
    logger.info(f"Data scraping complete. File saved as '{output_file}'.")

if __name__ == "__main__":
    main()


2024-09-30 23:44:27,880 - INFO - Starting the web scraping process.
2024-09-30 23:44:27,880 - INFO - Chrome WebDriver setup complete.
2024-09-30 23:44:27,880 - INFO - ====== WebDriver manager ======
2024-09-30 23:44:30,538 - INFO - Get LATEST chromedriver version for google-chrome
2024-09-30 23:44:30,872 - INFO - Get LATEST chromedriver version for google-chrome
2024-09-30 23:44:30,951 - INFO - Driver [C:\Users\santosh nandam\.wdm\drivers\chromedriver\win64\129.0.6668.70\chromedriver-win32/chromedriver.exe] found in cache
2024-09-30 23:44:39,288 - INFO - Loaded URL: https://www.tripadvisor.in/TravelersChoice-Hotels-cLuxury-g1
2024-09-30 23:44:39,288 - INFO - Scrolling through the page to load more hotels.
2024-09-30 23:44:41,347 - INFO - Scrolling through the page to load more hotels.
2024-09-30 23:44:43,396 - INFO - Reached the bottom of the page. Stopping scrolling.
2024-09-30 23:44:46,579 - INFO - Data scraping complete. File saved as 'tripadvisor_hotels_reviews.xlsx'.


In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import FileLink

# Function to scrape reviews from multiple hotels
def scrape_tripadvisor_reviews_multiple(start_urls, max_pages=5, output_file="tripadvisor_reviews_all_hotels.xlsx"):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
        "Accept-Language": "en-US,en;q=0.9",
        "Referer": "https://www.google.com/"
    }

    all_reviews_data = []  # List to store reviews for all hotels

    for start_url in start_urls:
        current_page = 0

        for page in range(1, max_pages + 1):
            # Construct the URL for each page based on TripAdvisor's pagination structure
            url = f"{start_url[:-5]}-or{(page-1)*10}{start_url[-5:]}"
            print(f"Fetching page {page} for hotel: {url}")

            # Send a GET request to fetch the webpage
            response = requests.get(url, headers=headers)

            # Check if the request was successful
            if response.status_code != 200:
                print(f"Failed to retrieve the page for {url}. Status code: {response.status_code}")
                break

            # Parse the page content with BeautifulSoup
            soup = BeautifulSoup(response.content, 'html.parser')

            # Extract hotel name (typically found in the page's main header)
            if page == 1:  # Hotel name is the same across pages, so we only need to extract it once
                hotel_name_tag = soup.find('h1')  # More flexible search for the hotel name
                if hotel_name_tag:
                    hotel_name = hotel_name_tag.get_text(strip=True)
                    print(f"Hotel Name: {hotel_name}")
                else:
                    hotel_name = "Hotel Name Not Found"
                    print("Could not find the hotel name.")

            # Find review containers (adjust the class as needed)
            review_divs = soup.find_all('div', class_='kmMXA _T Gi')

            if not review_divs:
                print(f"No reviews found on this page for hotel {hotel_name}.")
                break

            # Loop through all the found review divs and store their content in a list
            for idx, div in enumerate(review_divs, 1):
                review_text = div.get_text(strip=True)  # Extract review text

                # Extract review rating (Assuming it’s inside a span or div, adjust class if needed)
                rating_span = div.find('span', class_='ui_bubble_rating')
                if rating_span:
                    rating = rating_span['class'][1].split('_')[-1]  # Extract rating from class name
                    rating = int(rating) / 10  # Convert rating to a 1-5 scale
                else:
                    rating = None  # If no rating found

                # Extract reviewer name
                reviewer_name_tag = div.find('a', class_='ui_header_link')
                reviewer_name = reviewer_name_tag.get_text(strip=True) if reviewer_name_tag else "Anonymous"

                # Extract location
                location_tag = div.find('span', class_='default')
                location = location_tag.get_text(strip=True) if location_tag else "Location not available"

                # Append the review data
                all_reviews_data.append({
                    "Hotel Name": hotel_name,
                    "Location":location,
                    "Rating": rating,
                    "Reviewer Nmae":reviewer_name,
                    "Review Text": review_text
                })

            # Update the page counter
            current_page += 1

    # Check if any reviews were collected before creating a DataFrame
    if all_reviews_data:
        # Create a DataFrame from the list of reviews
        df = pd.DataFrame(all_reviews_data)

        # Write the DataFrame to an Excel file
        df.to_excel(output_file, index=False)
        print(f"All reviews successfully saved to {output_file}")
    else:
        print("No reviews were collected. No file created.")

# List of TripAdvisor hotel URLs
start_urls = [
    "https://www.tripadvisor.com/Hotel_Review-g1175545-d14184436-Reviews-Domes_Miramare_A_Luxury_Collection_Resort-Moraitika_Corfu_Ionian_Islands.html",
    "https://www.tripadvisor.com/Hotel_Review-g190384-d236327-Reviews-Olympic_Lagoon_Resorts_Paphos-Paphos_Paphos_District.html",
    "https://www.tripadvisor.com/Hotel_Review-g635613-d2017781-Reviews-Atrium_Platinum_Luxury_Resort_Hotel_Spa-Rhodes_Town_Rhodes_Dodecanese_South_Aegean.html",
    "https://www.tripadvisor.com/Hotel_Review-g190384-d267704-Reviews-Elysium_Hotel-Paphos_Paphos_District.html",
    "https://www.tripadvisor.com/Hotel_Review-g652080-d266645-Reviews-Vila_Vita_Parc_Resort_Spa-Porches_Faro_District_Algarve.html",
    "https://www.tripadvisor.com/Hotel_Review-g3236241-d253983-Reviews-Atrium_Palace_Thalasso_Spa_Resort_Villas-Kalathos_Rhodes_Dodecanese_South_Aegean.html",
    "https://www.tripadvisor.com/Hotel_Review-g189167-d229301-Reviews-Hotel_The_Cliff_Bay-Funchal_Madeira_Madeira_Islands.html",
    "https://www.tripadvisor.com/Hotel_Review-g293924-d20092115-Reviews-Peridot_Grand_Luxury_Boutique_Hotel-Hanoi.html",
    "https://www.tripadvisor.com/Hotel_Review-g293924-d17415144-Reviews-The_Oriental_Jade_Hotel-Hanoi.html",
    "https://www.tripadvisor.com/Hotel_Review-g187782-d228959-Reviews-Grand_Hotel_Excelsior_Vittoria-Sorrento_Province_of_Naples_Campania.html",
    "https://www.tripadvisor.com/Hotel_Review-g297704-d7355295-Reviews-InterContinental_Bandung_Dago_Pakar_an_IHG_hotel-Bandung_West_Java_Java.html",
    "https://www.tripadvisor.com/Hotel_Review-g295424-d3447941-Reviews-JW_Marriott_Marquis_Hotel_Dubai-Dubai_Emirate_of_Dubai.html",
    "https://www.tripadvisor.com/Hotel_Review-g662290-d282771-Reviews-Hotel_Fariones-Puerto_Del_Carmen_Tias_Lanzarote_Canary_Islands.html",
    "https://www.tripadvisor.com/Hotel_Review-g15300585-d557049-Reviews-Lara_Barut_Collection-Muratpasa_Antalya_Turkish_Mediterranean_Coast.html",
    "https://www.tripadvisor.com/Hotel_Review-g312658-d631305-Reviews-Atlanticview_Cape_Town_Boutique_Hotel-Camps_Bay_Western_Cape.html",
    "https://www.tripadvisor.com/Hotel_Review-g294201-d1597516-Reviews-Kempinski_Nile_Hotel_Cairo-Cairo_Cairo_Governorate.html",
    "https://www.tripadvisor.com/Hotel_Review-g60878-d21059395-Reviews-Lotte_Hotel_Seattle-Seattle_Washington.html",
    "https://www.tripadvisor.com/Hotel_Review-g609056-d300025-Reviews-Hotel_Aqua-Icmeler_Marmaris_District_Mugla_Province_Turkish_Aegean_Coast.html",
    "https://www.tripadvisor.com/Hotel_Review-g297712-d24883682-Reviews-Padma_Hotel_Semarang-Semarang_Central_Java_Java.html",
    "https://www.tripadvisor.com/Hotel_Review-g294265-d19888117-Reviews-The_Clan_Hotel_Singapore_by_Far_East_Hospitality-Singapore.html",
    "https://www.tripadvisor.com/Hotel_Review-g667803-d3226759-Reviews-Sunrise_Pearl_Hotel_Spa-Protaras_Paralimni_Famagusta_District.html",
    "https://www.tripadvisor.com/Hotel_Review-g304551-d3510307-Reviews-The_Leela_Ambience_Convention_Hotel_Delhi-New_Delhi_National_Capital_Territory_of_Delh.html",
    "https://www.tripadvisor.com/Hotel_Review-g293916-d13134217-Reviews-Akara_Hotel-Bangkok.html",
    "https://www.tripadvisor.com/Hotel_Review-g15516847-d1591768-Reviews-Premier_Le_Reve_Hotel_Spa-Sahl_Hasheesh_Hurghada_Red_Sea_and_Sinai.html",
    "https://www.tripadvisor.com/Hotel_Review-g303855-d1552134-Reviews-Rixos_Sharm_El_Sheikh-Nabq_Bay_Sharm_El_Sheikh_South_Sinai_Red_Sea_and_Sinai.html",
    "https://www.tripadvisor.com/Hotel_Review-g297712-d21302449-Reviews-Hotel_Tentrem_Semarang-Semarang_Central_Java_Java.html",
    "https://www.tripadvisor.com/Hotel_Review-g309274-d586762-Reviews-Gaia_Hotel_Nature_Reserve-Manuel_Antonio_Quepos_Province_of_Puntarenas.html",
    "https://www.tripadvisor.com/Hotel_Review-g295424-d2445260-Reviews-Rixos_The_Palm_Hotel_Suites-Dubai_Emirate_of_Dubai.html",
    "https://www.tripadvisor.com/Hotel_Review-g7382880-d230955-Reviews-Four_Seasons_Hotel-Agios_Tychon_Limassol.html",
    "https://www.tripadvisor.com/Hotel_Review-g189158-d195643-Reviews-Hotel_Avenida_Palace-Lisbon_Lisbon_District_Central_Portugal.html",
    "https://www.tripadvisor.com/Hotel_Review-g295424-d507352-Reviews-Grosvenor_House_A_Luxury_Collection_Hotel_Dubai-Dubai_Emirate_of_Dubai.html",
    "https://www.tripadvisor.com/Hotel_Review-g662606-d1141708-Reviews-JOIA_El_Mirador_by_Iberostar-Costa_Adeje_Adeje_Tenerife_Canary_Islands.html",
    "https://www.tripadvisor.com/Hotel_Review-g303855-d8447673-Reviews-Rixos_Premium_Seagate-Nabq_Bay_Sharm_El_Sheikh_South_Sinai_Red_Sea_and_Sinai.html",
    "https://www.tripadvisor.com/Hotel_Review-g2318557-d14943429-Reviews-Ivi_Mare_Designed_For_Adults_by_Louis_Hotels-Geroskipou_Paphos_District.html",
    "https://www.tripadvisor.com/Hotel_Review-g2318557-d284381-Reviews-Constantinou_Bros_Asimina_Suites_Hotel-Geroskipou_Paphos_District.html"
    "https://www.tripadvisor.com/Hotel_Review-g297549-d647102-Reviews-Steigenberger_ALDAU_Beach_Hotel-Hurghada_Red_Sea_and_Sinai.html",
    "https://www.tripadvisor.com/Hotel_Review-g186600-d213383-Reviews-Hayfield_Manor_Hotel-Cork_County_Cork.html",
    "https://www.tripadvisor.com/Hotel_Review-g939713-d12279503-Reviews-Ikos_Dassia-Dassia_Corfu_Ionian_Islands.html",
    "https://www.tripadvisor.com/Hotel_Review-g298342-d316750-Reviews-Constance_Belle_Mare_Plage-Belle_Mare.html",
    "https://www.tripadvisor.com/Hotel_Review-g295424-d302206-Reviews-The_Ritz_Carlton_Dubai-Dubai_Emirate_of_Dubai.html",
    "https://www.tripadvisor.com/Hotel_Review-g35805-d114591-Reviews-Four_Seasons_Hotel_Chicago-Chicago_Illinois.html",
    "https://www.tripadvisor.com/Hotel_Review-g190454-d206598-Reviews-Hotel_Sacher_Wien-Vienna.html",
    "https://www.tripadvisor.com/Hotel_Review-g147311-d15327038-Reviews-S_Hotel_Montego_Bay-Montego_Bay_Saint_James_Parish_Jamaica.html",
    "https://www.tripadvisor.com/Hotel_Review-g14782503-d3737584-Reviews-Hotel_Tentrem_Yogyakarta-Yogyakarta_Yogyakarta_Region_Java.html",
    "https://www.tripadvisor.com/Hotel_Review-g187493-d236219-Reviews-Hotel_Rector-Salamanca_Province_of_Salamanca_Castile_and_Leon.html",
    "https://www.tripadvisor.com/Hotel_Review-g189158-d12659702-Reviews-Corpo_Santo_Lisbon_Historical_Hotel-Lisbon_Lisbon_District_Central_Portugal.html",
    "https://www.tripadvisor.com/Hotel_Review-g294314-d304538-Reviews-Palacio_del_Inka_a_Luxury_Collection_Hotel_Cusco-Cusco_Cusco_Region.html",
    "https://www.tripadvisor.com/Hotel_Review-g274707-d507334-Reviews-Ventana_Hotel_Prague-Prague_Bohemia.html",
    "https://www.tripadvisor.com/Hotel_Review-g293940-d12828966-Reviews-Palace_Gate_Hotel_Resort-Phnom_Penh.html",
    "https://www.tripadvisor.com/Hotel_Review-g1893002-d302446-Reviews-Melia_Koh_Samui-Choeng_Mon_Bophut_Ko_Samui_Surat_Thani_Province.html",
    "https://www.tripadvisor.com/Hotel_Review-g298085-d16891462-Reviews-Sala_Danang_Beach_Hotel-Da_Nang.html",
    "https://www.tripadvisor.com/Hotel_Review-g187791-d205044-Reviews-Hotel_Artemide-Rome_Lazio.html",
    "https://www.tripadvisor.com/Hotel_Review-g660503-d7777995-Reviews-Seven_Seas_Hotel_Life-Goynuk_Kemer_Turkish_Mediterranean_Coast.html",
    "https://www.tripadvisor.com/Hotel_Review-g154943-d1063797-Reviews-Rosewood_Hotel_Georgia-Vancouver_British_Columbia.html",
    "https://www.tripadvisor.com/Hotel_Review-g293916-d301411-Reviews-Mandarin_Oriental_Bangkok-Bangkok.html",
    "https://www.tripadvisor.com/Hotel_Review-g186605-d208232-Reviews-The_Merrion_Hotel-Dublin_County_Dublin.html",
    "https://www.tripadvisor.com/Hotel_Review-g23068219-d5049935-Reviews-Lotte_Hotel_Hanoi-Ba_Dinh_Hanoi.html",
    "https://www.tripadvisor.com/Hotel_Review-g186338-d188961-Reviews-Hotel_41-London_England.html",
    "https://www.tripadvisor.com/Hotel_Review-g3336531-d15142069-Reviews-Clark_Marriott_Hotel-Clark_Freeport_Zone_Pampanga_Province_Central_Luzon_Region_Luzo.html",
    "https://www.tripadvisor.com/Hotel_Review-g294217-d548045-Reviews-Four_Seasons_Hotel_Hong_Kong-Hong_Kong.html",
    "https://www.tripadvisor.com/Hotel_Review-g293974-d1604061-Reviews-White_House_Hotel_Istanbul-Istanbul.html",
    "https://www.tripadvisor.com/Hotel_Review-g667803-d15558432-Reviews-Sunrise_Jade_Hotel-Protaras_Paralimni_Famagusta_District.html",
    "https://www.tripadvisor.com/Hotel_Review-g295424-d17659979-Reviews-JA_Lake_View_Hotel-Dubai_Emirate_of_Dubai.html",
    "https://www.tripadvisor.com/Hotel_Review-g910630-d15182081-Reviews-Ikos_Aria-Kefalos_Kos_Dodecanese_South_Aegean.html",
    "https://www.tripadvisor.com/Hotel_Review-g667803-d13074526-Reviews-The_Blue_Ivy_Hotel_Suites-Protaras_Paralimni_Famagusta_District.html",
    "https://www.tripadvisor.com/Hotel_Review-g190384-d292163-Reviews-King_Jason_Paphos_Designed_For_Adults_by_Louis_Hotels-Paphos_Paphos_District.html"
    "https://www.tripadvisor.com/Hotel_Review-g189158-d13402042-Reviews-Hotel_Da_Baixa-Lisbon_Lisbon_District_Central_Portugal.html",
    "https://www.tripadvisor.com/Hotel_Review-g186600-d625593-Reviews-The_Montenotte_Hotel-Cork_County_Cork.html",
    "https://www.tripadvisor.com/Hotel_Review-g1379324-d16711842-Reviews-InterContinental_Phuket_Resort_an_IHG_hotel-Kamala_Kathu_Phuket.html",
    "https://www.tripadvisor.com/Hotel_Review-g274887-d13392862-Reviews-Hotel_Clark_Budapest-Budapest_Central_Hungary.html",
    "https://www.tripadvisor.com/Hotel_Review-g293924-d17627438-Reviews-La_Sinfonia_del_Rey_Hotel_Spa-Hanoi.html",
    "https://www.tripadvisor.com/Hotel_Review-g293924-d615317-Reviews-La_Siesta_Premium_Hang_Be_Hotel-Hanoi.html",
    "https://www.tripadvisor.com/Hotel_Review-g293924-d9808263-Reviews-Bespoke_Trendy_Hotel_Hanoi-Hanoi.html",
    "https://www.tripadvisor.com/Hotel_Review-g293974-d8364987-Reviews-Romance_Istanbul_Hotel-Istanbul.html",
    "https://www.tripadvisor.com/Hotel_Review-g274707-d658737-Reviews-Hotel_Residence_Agnes-Prague_Bohemia.html",
    "https://www.tripadvisor.com/Hotel_Review-g187147-d617625-Reviews-Grand_Hotel_Du_Palais_Royal-Paris_Ile_de_France.html",
    "https://www.tripadvisor.com/Hotel_Review-g274887-d15124757-Reviews-Mystery_Hotel_Budapest-Budapest_Central_Hungary.html",
    "https://www.tripadvisor.com/Hotel_Review-g190384-d291037-Reviews-Alexander_The_Great_Beach_Hotel-Paphos_Paphos_District.html",
    "https://www.tripadvisor.com/Hotel_Review-g188671-d206720-Reviews-Hotel_Heritage_Relais_Chateaux-Bruges_West_Flanders_Province.html",
    "https://www.tripadvisor.com/Hotel_Review-g293924-d10550496-Reviews-O_Gallery_Premier_Hotel_Spa-Hanoi.html",
    "https://www.tripadvisor.com/Hotel_Review-g580312-d573632-Reviews-Hotel_Astoria_Playa_Only_Adults-Port_d_Alcudia_Alcudia_Majorca_Balearic_Islands.html",
    "https://www.tripadvisor.com/Hotel_Review-g635626-d229044-Reviews-Grand_Hotel_Tremezzo-Tremezzina_Lake_Como_Lombardy.html",
    "https://www.tripadvisor.com/Hotel_Review-g293980-d301856-Reviews-Herods_Vitalis_Spa_Hotel_Eilat-Eilat_Southern_District.html",
    "https://www.tripadvisor.com/Hotel_Review-g297550-d600103-Reviews-Jaz_Makadi_Star_Spa-Makadi_Bay_Hurghada_Red_Sea_and_Sinai.html",
    "https://www.tripadvisor.com/Hotel_Review-g488095-d1308172-Reviews-Kempinski_Hotel_Soma_Bay-Soma_Bay_Red_Sea_and_Sinai.html",
    "https://www.tripadvisor.com/Hotel_Review-g10804710-d12629455-Reviews-Avista_Grande_Phuket_Karon_MGallery_Hotel_Collection-Karon_Beach_Karon_Phuket.html",
    "https://www.tripadvisor.com/Hotel_Review-g667803-d482620-Reviews-Crystal_Springs_Beach_Hotel-Protaras_Paralimni_Famagusta_District.html",
    "https://www.tripadvisor.com/Hotel_Review-g10006284-d972924-Reviews-Seven_Stars_Resort_Spa-Grace_Bay_Providenciales_Turks_and_Caicos.html",
    "https://www.tripadvisor.com/Hotel_Review-g295424-d8631865-Reviews-Al_Habtoor_Palace_Preferred_Hotels_Resorts-Dubai_Emirate_of_Dubai.html",
    "https://www.tripadvisor.com/Hotel_Review-g187782-d1656729-Reviews-Grand_Hotel_La_Favorita-Sorrento_Province_of_Naples_Campania.html",
    "https://www.tripadvisor.com/Hotel_Review-g293924-d7180030-Reviews-La_Siesta_Classic_Ma_May_Hotel-Hanoi.html",
    "https://www.tripadvisor.com/Hotel_Review-g608947-d278215-Reviews-AX_ODYCY_Hotel-Qawra_Island_of_Malta.html",
    "https://www.tripadvisor.com/Hotel_Review-g35805-d225105-Reviews-The_Peninsula_Chicago-Chicago_Illinois.html",
    "https://www.tripadvisor.com/Hotel_Review-g293924-d1546077-Reviews-Splendid_Star_Grand_Hotel_Spa-Hanoi.html",
    "https://www.tripadvisor.com/Hotel_Review-g297390-d9514361-Reviews-Viroth_s_Hotel-Siem_Reap_Siem_Reap_Province.html",
    "https://www.tripadvisor.com/Hotel_Review-g274707-d277304-Reviews-Golden_Well_Hotel-Prague_Bohemia.html",
    "https://www.tripadvisor.com/Hotel_Review-g186338-d188019-Reviews-Brown_s_Hotel-London_England.html",
    "https://www.tripadvisor.com/Hotel_Review-g32070-d76020-Reviews-The_Peninsula_Beverly_Hills-Beverly_Hills_California.html",
    "https://www.tripadvisor.com/Hotel_Review-g187895-d195006-Reviews-Hotel_Lungarno-Florence_Tuscany.html",
    "https://www.tripadvisor.com/Hotel_Review-g298082-d13378296-Reviews-Allegro_Hoi_An_A_Little_Luxury_Hotel_Spa-Hoi_An_Quang_Nam_Province.html",
    "https://www.tripadvisor.com/Hotel_Review-g659660-d243098-Reviews-Palladium_Boutique_Hotel-Platys_Gialos_Mykonos_Cyclades_South_Aegean.html",
    "https://www.tripadvisor.com/Hotel_Review-g635864-d471829-Reviews-Santorini_Princess_Spa_Hotel-Imerovigli_Santorini_Cyclades_South_Aegean.html",
    "https://www.tripadvisor.com/Hotel_Review-g503693-d1171944-Reviews-Grand_View_Hotel-Dalhousie_Chamba_District_Himachal_Pradesh.html",
    "https://www.tripadvisor.com/Hotel_Review-g60956-d6884400-Reviews-Hotel_Emma-San_Antonio_Texas.html",
    "https://www.tripadvisor.com/Hotel_Review-g297930-d13140255-Reviews-Hotel_Clover_Patong_Phuket-Patong_Kathu_Phuket.html",
    "https://www.tripadvisor.com/Hotel_Review-g187069-d19566904-Reviews-Stock_Exchange_Hotel-Manchester_Greater_Manchester_England.html",
    "https://www.tripadvisor.com/Hotel_Review-g298082-d25173194-Reviews-Hoi_An_Delicacy_Hotel_Spa-Hoi_An_Quang_Nam_Province.html",
    "https://www.tripadvisor.com/Hotel_Review-g274887-d9250207-Reviews-Hotel_Moments_Budapest-Budapest_Central_Hungary.html",
    "https://www.tripadvisor.com/Hotel_Review-g297683-d302143-Reviews-The_Oberoi_Amarvilas-Agra_Agra_District_Uttar_Pradesh.html",
    "https://www.tripadvisor.com/Hotel_Review-g295424-d302098-Reviews-Shangri_La_Dubai-Dubai_Emirate_of_Dubai.html",
    "https://www.tripadvisor.com/Hotel_Review-g60763-d15135187-Reviews-Park_Terrace_Hotel-New_York_City_New_York.html",
    "https://www.tripadvisor.com/Hotel_Review-g187791-d13393444-Reviews-Singer_Palace_Hotel_Roma-Rome_Lazio.html",
    "https://www.tripadvisor.com/Hotel_Review-g488100-d316741-Reviews-Paradise_Cove_Boutique_Hotel-Anse_La_Raie.html",
    "https://www.tripadvisor.com/Hotel_Review-g295424-d21275006-Reviews-Al_Jaddaf_Rotana_Suite_Hotel-Dubai_Emirate_of_Dubai.html",
    "https://www.tripadvisor.com/Hotel_Review-g293797-d4375055-Reviews-Kempinski_Hotel_Gold_Coast_City-Accra_Greater_Accra.html",
    "https://www.tripadvisor.com/Hotel_Review-g274887-d7818402-Reviews-Prestige_Hotel_Budapest-Budapest_Central_Hungary.html",
    "https://www.tripadvisor.com/Hotel_Review-g20361178-d229400-Reviews-Parklane_a_Luxury_Collection_Resort_Spa_Limassol-Pyrgos_Limassol.html",
    "https://www.tripadvisor.com/Hotel_Review-g312725-d499754-Reviews-Xanadu_Resort_Hotel-Belek_Serik_District_Turkish_Mediterranean_Coast.html",
    "https://www.tripadvisor.com/Hotel_Review-g295424-d21511241-Reviews-Avani_Palm_View_Dubai_Hotel_Suites-Dubai_Emirate_of_Dubai.html",
    "https://www.tripadvisor.com/Hotel_Review-g298573-d1197763-Reviews-Admiral_Hotel_Manila_MGallery-Manila_Metro_Manila_Luzon.html",
    "https://www.tripadvisor.com/Hotel_Review-g152515-d15752722-Reviews-Nobu_Hotel_Los_Cabos-Cabo_San_Lucas_Los_Cabos_Baja_California.html",
    "https://www.tripadvisor.com/Hotel_Review-g293924-d15131972-Reviews-La_Sinfonia_Citadel_Hotel_And_Spa-Hanoi.html",
    "https://www.tripadvisor.com/Hotel_Review-g608524-d15039915-Reviews-The_Golden_Ridge_Hotel-Nuwara_Eliya_Central_Province.html",
    "https://www.tripadvisor.com/Hotel_Review-g293916-d301388-Reviews-Montien_Hotel_Surawong_Bangkok-Bangkok.html",
    "https://www.tripadvisor.com/Hotel_Review-g187443-d471980-Reviews-Hotel_Gravina_51-Seville_Province_of_Seville_Andalucia.html",
    "https://www.tripadvisor.com/Hotel_Review-g293890-d15530725-Reviews-Kathmandu_Marriott_Hotel-Kathmandu_Kathmandu_Valley_Bagmati_Zone_Central_Region.html",
    "https://www.tripadvisor.com/Hotel_Review-g1182872-d15125547-Reviews-Salt_Of_Palmar_Mauritius_A_Member_Of_Design_Hotels-Palmar.html",
    "https://www.tripadvisor.com/Hotel_Review-g295424-d14476134-Reviews-Hotel_Indigo_Dubai_Downtown_an_IHG_hotel-Dubai_Emirate_of_Dubai.html",
    "https://www.tripadvisor.com/Hotel_Review-g293916-d302456-Reviews-The_Athenee_Hotel_a_Luxury_Collection_Hotel_Bangkok-Bangkok.html"
    "https://www.tripadvisor.com/Hotel_Review-g187514-d228529-Reviews-Palacio_de_los_Duques_Gran_Melia_The_Leading_Hotels_of_the_World-Madrid.html",
    "https://www.tripadvisor.com/Hotel_Review-g187147-d9452312-Reviews-Le_Narcisse_Blanc_Hotel_Spa-Paris_Ile_de_France.html",
    "https://www.tripadvisor.in/Hotel_Review-g297715-d6883351-Reviews-Hotel_Ciputra_World_Surabaya-Surabaya_East_Java_Java.html",
    "https://www.tripadvisor.in/Hotel_Review-g274772-d2140867-Reviews-Queen_Boutique_Hotel-Krakow_Lesser_Poland_Province_Southern_Poland.html",
    "https://www.tripadvisor.in/Hotel_Review-g295424-d23631662-Reviews-The_First_Collection_At_Jumeirah_Village_Circle_A_Tribute_Portfolio_Hotel-Dubai_Emira.html",
    "https://www.tripadvisor.in/Hotel_Review-g666312-d2508715-Reviews-Aqua_Blu_Boutique_Hotel_Spa-Kos_Town_Kos_Dodecanese_South_Aegean.html",
    "https://www.tripadvisor.in/Hotel_Review-g187895-d203921-Reviews-Hotel_David-Florence_Tuscany.html",
    "https://www.tripadvisor.in/Hotel_Review-g294191-d15015983-Reviews-Wyndham_Grand_Yangon_Hotel-Yangon_Rangoon_Yangon_Region.html",
    "https://www.tripadvisor.in/Hotel_Review-g194867-d1062501-Reviews-Hotel_Pellegrino-Praiano_Amalfi_Coast_Province_of_Salerno_Campania.html",
    "https://www.tripadvisor.in/Hotel_Review-g303444-d313150-Reviews-Hotel_das_Cataratas_A_Belmond_Hotel_Iguassu_Falls-Foz_do_Iguacu_State_of_Parana.html"
    "https://www.tripadvisor.in/Hotel_Review-g227884-d1017752-Reviews-Lindos_Blu_Luxury_Hotel_Suites-Lindos_Rhodes_Dodecanese_South_Aegean.html",
    "https://www.tripadvisor.in/Hotel_Review-g658914-d235731-Reviews-Cavo_Bianco_Boutique_Hotel_Spa-Kamari_Santorini_Cyclades_South_Aegean.html",
    "https://www.tripadvisor.in/Hotel_Review-g295424-d23798830-Reviews-Premier_Inn_Dubai_Barsha_Heights_Hotel-Dubai_Emirate_of_Dubai.html",
    "https://www.tripadvisor.in/Hotel_Review-g150793-d6002592-Reviews-Hotel_Mousai_Puerto_Vallarta-Puerto_Vallarta.html",
    "https://www.tripadvisor.in/Hotel_Review-g303833-d578554-Reviews-Monte_Mulini_Adults_Exclusive_Hotel_by_Maistra_Collection-Rovinj_Istria.html",
    "https://www.tripadvisor.in/Hotel_Review-g293924-d19699550-Reviews-Solaria_Hotel-Hanoi.html",
    "https://www.tripadvisor.in/Hotel_Review-g679021-d24981677-Reviews-Alaukik_Hotel_Shirdi-Shirdi_Ahmednagar_District_Maharashtra.html",
    "https://www.tripadvisor.in/Hotel_Review-g293924-d26362788-Reviews-La_Siesta_Classic_Hang_Thung_Hotel-Hanoi.html",
    "https://www.tripadvisor.in/Hotel_Review-g187514-d10190864-Reviews-Gran_Hotel_Ingles-Madrid.html",
    "https://www.tripadvisor.in/Hotel_Review-g482941-d2550137-Reviews-Andronis_Boutique_Hotel-Oia_Santorini_Cyclades_South_Aegean.html",
    "https://www.tripadvisor.in/Hotel_Review-g1182872-d15125547-Reviews-Salt_Of_Palmar_Mauritius_A_Member_Of_Design_Hotels-Palmar.html"
    # Add more URLs as needed
]
# Call the function with a list of start URLs
# Scrape and save reviews for multiple hotels to an Excel file
scrape_tripadvisor_reviews_multiple(start_urls, max_pages=5, output_file="tripadvisor_reviews_all_hotels.xlsx")
FileLink('tripadvisor_reviews_all_hotels.xlsx')

Fetching page 1 for hotel: https://www.tripadvisor.com/Hotel_Review-g1175545-d14184436-Reviews-Domes_Miramare_A_Luxury_Collection_Resort-Moraitika_Corfu_Ionian_Islands-or0.html
Failed to retrieve the page for https://www.tripadvisor.com/Hotel_Review-g1175545-d14184436-Reviews-Domes_Miramare_A_Luxury_Collection_Resort-Moraitika_Corfu_Ionian_Islands-or0.html. Status code: 403
Fetching page 1 for hotel: https://www.tripadvisor.com/Hotel_Review-g190384-d236327-Reviews-Olympic_Lagoon_Resorts_Paphos-Paphos_Paphos_District-or0.html
Failed to retrieve the page for https://www.tripadvisor.com/Hotel_Review-g190384-d236327-Reviews-Olympic_Lagoon_Resorts_Paphos-Paphos_Paphos_District-or0.html. Status code: 403
Fetching page 1 for hotel: https://www.tripadvisor.com/Hotel_Review-g635613-d2017781-Reviews-Atrium_Platinum_Luxury_Resort_Hotel_Spa-Rhodes_Town_Rhodes_Dodecanese_South_Aegean-or0.html
Failed to retrieve the page for https://www.tripadvisor.com/Hotel_Review-g635613-d2017781-Reviews-Atrium_Pl

KeyboardInterrupt: 